In [1]:
## based on 2_training and 3_prediction from StarDist

In [2]:
import inspect
import pickle

In [3]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tqdm import tqdm
import tifffile as tif
from csbdeep.utils import Path, normalize

from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available,_draw_polygons
from stardist.models import Config2D, StarDist2D, StarDistData2D

np.random.seed(42)
lbl_cmap = random_label_cmap()

    
from csbdeep.utils.tf import limit_gpu_memory
# adjust as necessary: limit GPU memory to be used by TensorFlow to leave some to OpenCL-based computations
limit_gpu_memory(0.8)

import os

Using TensorFlow backend.
/home/trasse/anaconda3/envs/opsef004/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/trasse/anaconda3/envs/opsef004/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/trasse/anaconda3/envs/opsef004/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/trasse/a

# Data

<div class="alert alert-block alert-info">
Training data (for input `X` with associated label masks `Y`) can be provided via lists of numpy arrays, where each image can have a different size. Alternatively, a single numpy array can also be used if all images have the same size.  
Input images can either be two-dimensional (single-channel) or three-dimensional (multi-channel) arrays, where the channel axis comes last. Label images need to be integer-valued.
</div>

In [4]:
main_folder = os.path.dirname(os.path.abspath(inspect.stack()[0][1])).replace("TrainStarDist","Augment")
file_path = "{}/my_runs/augment_settings_xl.pkl".format(main_folder)
infile = open(file_path,'rb')
parameter = pickle.load(infile)
print("Loading processing pipeline from",file_path)
infile.close()
aug_sets,pre_defined_pipelines,data_main_GT,Datasets_Download = parameter

Loading processing pipeline from /home/trasse/github/OpSeF-IV/Train/Augment/my_runs/augment_settings_xl.pkl


In [5]:
run_list = []

In [6]:
# define training 1
trainModelSettings = {}
trainModelSettings["root"] = data_main_GT
trainModelSettings["data"] = "DSB2018_FL_Nuc_Subset"
trainModelSettings["path"] = os.path.join(trainModelSettings["root"],trainModelSettings["data"])
trainModelSettings["basedir_StarDist_Train"] = "./models"
trainModelSettings["name"] = 'Non_Augmented_Train_Small2'
trainModelSettings["nrays"] = 32
trainModelSettings["epochs"] = 4
trainModelSettings["steps_per_epoch"] = 100
trainModelSettings["train_patch_size"] = (256,256)
trainModelSettings["unet_activation"]= 'relu'

run_list.append(trainModelSettings)

In [7]:
# define training 2
trainModelSettings = {}
trainModelSettings["root"] = data_main_GT
trainModelSettings["data"] = "DSB2018_FL_Nuc_Subset"
trainModelSettings["path"] = os.path.join(trainModelSettings["root"],trainModelSettings["data"])
trainModelSettings["basedir_StarDist_Train"] = "./models"
trainModelSettings["name"] = 'Non_Augmented_Train_Medium2'
trainModelSettings["nrays"] = 32
trainModelSettings["epochs"] = 4
trainModelSettings["steps_per_epoch"] = 100
trainModelSettings["train_patch_size"] = (256,256)
trainModelSettings["unet_activation"]= 'relu'

run_list.append(trainModelSettings)

In [8]:
def config_stardist(tms,n_channel):
    # configures StarDist based on the standart settings and the trainModelSettings
    n_rays = trainModelSettings["nrays"]
    use_gpu = False and gputools_available()
    grid = (2,2)    
    conf = Config2D (
        n_rays       = trainModelSettings["nrays"],
        grid         = grid,
        use_gpu      = use_gpu,
        n_channel_in = n_channel,
        train_patch_size = trainModelSettings["train_patch_size"],
        unet_activation = trainModelSettings["unet_activation"],
    )
    print(conf)
    vars(conf)
    return conf

In [9]:
for trainModelSettings in run_list:
    # prepare data
    X = sorted(glob('{}/train/images/*.tif'.format(trainModelSettings["path"])))
    Y = sorted(glob('{}/train/masks/*.tif'.format(trainModelSettings["path"])))
    assert all(Path(x).name==Path(y).name for x,y in zip(X,Y))
    
    # load data
    X = list(map(tif.imread,X))
    Y = list(map(tif.imread,Y))
    n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
    
    # Normalize images and fill small label holes
    axis_norm = (0,1)   # normalize channels independently
    # axis_norm = (0,1,2) # normalize channels jointly
    if n_channel > 1:
        print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
        sys.stdout.flush()
    X = [normalize(x,1,99.8,axis=axis_norm) for x in tqdm(X)]
    Y = [fill_label_holes(y) for y in tqdm(Y)]
    
    # Split into train and validation datasets.
    assert len(X) > 1, "not enough training data"
    rng = np.random.RandomState(42)
    ind = rng.permutation(len(X))
    n_val = max(1, int(round(0.15 * len(ind))))
    ind_train, ind_val = ind[:-n_val], ind[-n_val:]
    X_val, Y_val = [X[i] for i in ind_val]  , [Y[i] for i in ind_val]
    X_trn, Y_trn = [X[i] for i in ind_train], [Y[i] for i in ind_train] 
    print('number of images: %3d' % len(X))
    print('- training:       %3d' % len(X_trn))
    print('- validation:     %3d' % len(X_val))
    
    # define setting
    conf = config_stardist(trainModelSettings,n_channel)
    augmenter = None

    # train model
    model = StarDist2D(conf, name=trainModelSettings["name"], basedir = trainModelSettings["basedir_StarDist_Train"])
    median_size = calculate_extents(list(Y), np.median)
    fov = np.array(model._axes_tile_overlap('YX'))
    if any(median_size > fov):
        print("WARNING: median object size larger than field of view of the neural network.")
    model.train(X_trn, Y_trn, validation_data=(X_val,Y_val), augmenter=augmenter,
                epochs=trainModelSettings["epochs"] , steps_per_epoch=trainModelSettings["steps_per_epoch"])
    model.optimize_thresholds(X_val, Y_val)

100%|██████████| 77/77 [00:01<00:00, 66.85it/s]
/home/trasse/anaconda3/envs/opsef004/lib/python3.6/site-packages/csbdeep/models/base_model.py:134: UserWarning: output path for model already exists, files may be overwritten: /home/trasse/github/OpSeF-IV/Train/TrainStarDist/models/Non_Augmented_Train_Small2
  warnings.warn('output path for model already exists, files may be overwritten: %s' % str(self.logdir.resolve()))


number of images:  77
- training:        65
- validation:      12
Config2D(axes='YXC', backbone='unet', grid=(2, 2), n_channel_in=1, n_channel_out=33, n_dim=2, n_rays=32, net_conv_after_unet=128, net_input_shape=(None, None, 1), net_mask_shape=(None, None, 1), train_background_reg=0.0001, train_batch_size=4, train_checkpoint='weights_best.h5', train_checkpoint_epoch='weights_now.h5', train_checkpoint_last='weights_last.h5', train_completion_crop=32, train_dist_loss='mae', train_epochs=400, train_foreground_only=0.9, train_learning_rate=0.0003, train_loss_weights=(1, 0.2), train_n_val_patches=None, train_patch_size=(256, 256), train_reduce_lr={'factor': 0.5, 'patience': 40, 'min_delta': 0}, train_shape_completion=False, train_steps_per_epoch=100, train_tensorboard=True, unet_activation='relu', unet_batch_norm=False, unet_dropout=0.0, unet_kernel_size=(3, 3), unet_last_activation='relu', unet_n_conv_per_depth=2, unet_n_depth=3, unet_n_filter_base=32, unet_pool=(2, 2), unet_prefix='', use

NMS threshold = 0.5:  80%|████████  | 16/20 [00:17<00:04,  1.12s/it, 0.482 -> 0.520]


Using optimized values: prob_thresh=0.473643, nms_thresh=0.3.
Saving to 'thresholds.json'.


100%|██████████| 77/77 [00:01<00:00, 64.04it/s]
/home/trasse/anaconda3/envs/opsef004/lib/python3.6/site-packages/csbdeep/models/base_model.py:134: UserWarning: output path for model already exists, files may be overwritten: /home/trasse/github/OpSeF-IV/Train/TrainStarDist/models/Non_Augmented_Train_Medium2
  warnings.warn('output path for model already exists, files may be overwritten: %s' % str(self.logdir.resolve()))


number of images:  77
- training:        65
- validation:      12
Config2D(axes='YXC', backbone='unet', grid=(2, 2), n_channel_in=1, n_channel_out=33, n_dim=2, n_rays=32, net_conv_after_unet=128, net_input_shape=(None, None, 1), net_mask_shape=(None, None, 1), train_background_reg=0.0001, train_batch_size=4, train_checkpoint='weights_best.h5', train_checkpoint_epoch='weights_now.h5', train_checkpoint_last='weights_last.h5', train_completion_crop=32, train_dist_loss='mae', train_epochs=400, train_foreground_only=0.9, train_learning_rate=0.0003, train_loss_weights=(1, 0.2), train_n_val_patches=None, train_patch_size=(256, 256), train_reduce_lr={'factor': 0.5, 'patience': 40, 'min_delta': 0}, train_shape_completion=False, train_steps_per_epoch=100, train_tensorboard=True, unet_activation='relu', unet_batch_norm=False, unet_dropout=0.0, unet_kernel_size=(3, 3), unet_last_activation='relu', unet_n_conv_per_depth=2, unet_n_depth=3, unet_n_filter_base=32, unet_pool=(2, 2), unet_prefix='', use

NMS threshold = 0.5:  75%|███████▌  | 15/20 [00:18<00:06,  1.24s/it, 0.568 -> 0.544]


Using optimized values: prob_thresh=0.484394, nms_thresh=0.3.
Saving to 'thresholds.json'.


In [10]:
jkjk

NameError: name 'jkjk' is not defined

# Configuration

A `StarDist2D` model is specified via a `Config2D` object.

In [ ]:
print(Config2D.__doc__)

**Note:** The trained `StarDist2D` model will *not* predict completed shapes for partially visible objects at the image boundary if `train_shape_completion=False` (which is the default option).

Check if the neural network has a large enough field of view to see up to the boundary of most objects.

# Data

We now load images from the sub-folder `test` that have not been used during training.

In [ ]:
X = sorted(glob('{}/test/images/*.tif'.format(trainModelSettings["root"])))
X = list(map(tif.imread,X))

n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))

In [ ]:
# show all test images
if False:
    fig, ax = plt.subplots(7,8, figsize=(16,16))
    for i,(a,x) in enumerate(zip(ax.flat, X)):
        a.imshow(x if x.ndim==2 else x[...,0], cmap='gray')
        a.set_title(i)
    [a.axis('off') for a in ax.flat]
    plt.tight_layout()
None;

## Prediction

Make sure to normalize the input image beforehand or supply a `normalizer` to the prediction function.

Calling `model.predict_instances` will
- predict object probabilities and star-convex polygon distances (see `model.predict` if you want those)
- perform non-maximum suppression (with overlap threshold `nms_thresh`) for polygons above object probability threshold `prob_thresh`.
- render all remaining polygon instances in a label image
- return the label instances image and also the details (coordinates, etc.) of all remaining polygons

In [ ]:
img = normalize(X[16], 1,99.8, axis=axis_norm)
labels, details = model.predict_instances(img)

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(img if img.ndim==2 else img[...,0], clim=(0,1), cmap='gray')
plt.imshow(labels, cmap=lbl_cmap, alpha=0.5)
plt.axis('off');

# Example results

In [ ]:
def example(model, i, show_dist=True):
    img = normalize(X[i], 1,99.8, axis=axis_norm)
    labels, details = model.predict_instances(img)

    plt.figure(figsize=(13,10))
    img_show = img if img.ndim==2 else img[...,0]
    coord, points, prob = details['coord'], details['points'], details['prob']
    plt.subplot(121); plt.imshow(img_show, cmap='gray'); plt.axis('off')
    a = plt.axis()
    _draw_polygons(coord, points, prob, grid=model.config.grid, show_dist=show_dist)
    plt.axis(a)
    plt.subplot(122); plt.imshow(img_show, cmap='gray'); plt.axis('off')
    plt.imshow(labels, cmap=lbl_cmap, alpha=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
example(model, 42)

In [ ]:
example(model, 15, False)